In [1]:
import cupy as cp
import cupyx
import scanpy as sc
import voyagerpy as vp
from cupyx.scipy.sparse import csr_matrix
import os

In [2]:
path = "/data/kanferg/Sptial_Omics/playGround/Data/Xenium/output_temp"
pathout = "/data/kanferg/Sptial_Omics/SpatialOmicsToolkit/out_1"
FilePrefix = "_072824" 

In [3]:
andata = sc.read_h5ad(os.path.join(pathout, "andata_save.h5ad"))

In [4]:
npzfile = cp.load(os.path.join(pathout, "obsp_distances.npz"))
data = cp.array(npzfile['data'])
indices = cp.array(npzfile['indices'])
indptr = cp.array(npzfile['indptr'])
shape = tuple(npzfile['shape'])

# Reconstruct the sparse matrix
sparse_matrix_distances = csr_matrix((data, indices, indptr), shape=shape)


npzfile = cp.load(os.path.join(pathout, "connectivities.npz"))
data = cp.array(npzfile['data'])
indices = cp.array(npzfile['indices'])
indptr = cp.array(npzfile['indptr'])
shape = tuple(npzfile['shape'])

# Reconstruct the sparse matrix
sparse_matrix_connectivities = csr_matrix((data, indices, indptr), shape=shape)

In [5]:
andata.obsp['distances'] = sparse_matrix_distances.get()
andata.obsp['connectivities'] = sparse_matrix_connectivities.get()

# andata.obsp['distances'] = sparse_matrix_distances
# andata.obsp['connectivities'] = sparse_matrix_connectivities

In [6]:
sc.pp.subsample(andata,copy=False,n_obs=10_000)

In [7]:
dist = andata.obsp['distances']
#dist = andata.obsp[f'distances'].copy()
epsilon = 1e-10
dist.data = 1 / (dist.data + epsilon)
dist /= dist.sum(axis=1)

In [9]:
andata.obsp["weights"] = dist

In [10]:
del dist
knn_graph = "weights"
andata.obsp[f"connectivities"] = (andata.obsp[knn_graph] > 0).astype(cp.float32)  